In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import Orange
from Orange.data import Domain, DiscreteVariable, ContinuousVariable
from orangecontrib.associate.fpgrowth import *
%matplotlib inline

In [ ]:
# Load and filter data

In [3]:
cs_mba = pd.read_excel(io=r'Online Retail.xlsx')
cs_mba_uk = cs_mba[cs_mba.Country == 'United Kingdom']

cs_mba_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Data clean by removing returned items

In [4]:
# remove returned items
cs_mba_uk = cs_mba_uk[~(cs_mba_uk.InvoiceNo.str.contains('C') == True)]
cs_mba_uk = cs_mba_uk[~cs_mba_uk.Quantity < 0]

cs_mba_uk.shape

(486286, 8)

In [5]:
cs_mba_uk.InvoiceNo.value_counts().shape
# Returns object containing counts of unique values.

(18786,)

## Create transactional database

In [7]:
items = list(cs_mba_uk.Description.unique())

grouped = cs_mba_uk.groupby('InvoiceNo')
transaction_level_df_uk = grouped.aggregate(lambda x: tuple(x)).reset_index()[['InvoiceNo','Description']]


In [16]:
transaction_dict = {item: 0 for item in items}
output_dict = dict()
temp = dict()

for record in transaction_level_df_uk.to_dict('records'):
    invoice_num = record['InvoiceNo']
    items_list = record['Description']
    transaction_dict = {item: 0 for item in items}
    transaction_dict.update({item: 1 for item in items if item in items_list})
    temp.update({invoice_num: transaction_dict})

new = [v for k,v in temp.items()]
transaction_df = pd.DataFrame(new)
del(transaction_df[transaction_df.columns[0]])
    

In [17]:
transaction_df.shape

(18786, 4058)

In [18]:
transaction_df.head()

,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,5 HOOK HANGER MAGIC TOADSTOOL,60 TEATIME FAIRY CAKE CASES,"AIRLINE LOUNGE,METAL SIGN",ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE RED,ANTIQUE GLASS DRESSING TABLE POT,...,SET OF 6 RIBBONS COUNTRY STYLE,SNACK TRAY RED VINTAGE DOILY,SET OF 6 RIBBONS PERFECTLY PRETTY,SET OF 6 RIBBONS PARTY,SET 10 CARDS SNOWY ROBIN 17099,SET 10 CARDS SWIRLY XMAS TREE 17104,"LETTER ""U"" BLING KEY RING",CREAM HANGING HEART T-LIGHT HOLDER,BLACK SIL'T SQU CANDLE PLATE,"PAPER CRAFT , LITTLE BIRDIE"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Prune Dataset for frequently purchased items

In [19]:
def prune_dataset(input_df, length_trans = 2, total_sales_perc = 0.5, start_item = None, end_item = None):
    if 'total_items' in input_df.columns:
        del(input_df['total_items'])
    item_count = input_df.sum().sort_values(ascending = False).reset_index()
    total_items = sum(input_df.sum().sort_values(ascending = False))
    item_count.rename(columns={item_count.columns[0]:'item_name',item_count.columns[1]:'item_count'}, inplace=True)
    if not start_item and not end_item: 
        item_count['item_perc'] = item_count['item_count']/total_items
        item_count['total_perc'] = item_count.item_perc.cumsum()
        selected_items = list(item_count[item_count.total_perc < total_sales_perc].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items], item_count[item_count.total_perc < total_sales_perc]
    elif end_item > start_item:
        selected_items = list(item_count[start_item:end_item].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items],item_count[start_item:end_item]

In [21]:
output_df_uk, item_counts = prune_dataset(input_df = transaction_df, 
                                          length_trans = 2,
                                          start_item=0, 
                                          end_item=15)
print(output_df_uk.shape)
print(list(output_df_uk.columns))

#we have only 15 items responsible for 50% of sales 
# and 4961 transactions that have those items along with other items

(4961, 15)
['WHITE HANGING HEART T-LIGHT HOLDER', 'JUMBO BAG RED RETROSPOT', 'REGENCY CAKESTAND 3 TIER', 'PARTY BUNTING', 'LUNCH BAG RED RETROSPOT', 'ASSORTED COLOUR BIRD ORNAMENT', 'SET OF 3 CAKE TINS PANTRY DESIGN ', 'NATURAL SLATE HEART CHALKBOARD ', 'LUNCH BAG  BLACK SKULL.', 'HEART OF WICKER SMALL', 'JUMBO BAG PINK POLKADOT', 'JUMBO SHOPPER VINTAGE RED PAISLEY', 'JUMBO STORAGE BAG SUKI', 'PACK OF 72 RETROSPOT CAKE CASES', "PAPER CHAIN KIT 50'S CHRISTMAS "]


In [22]:
output_df_uk.head()

,WHITE HANGING HEART T-LIGHT HOLDER,JUMBO BAG RED RETROSPOT,REGENCY CAKESTAND 3 TIER,PARTY BUNTING,LUNCH BAG RED RETROSPOT,ASSORTED COLOUR BIRD ORNAMENT,SET OF 3 CAKE TINS PANTRY DESIGN,NATURAL SLATE HEART CHALKBOARD,LUNCH BAG BLACK SKULL.,HEART OF WICKER SMALL,JUMBO BAG PINK POLKADOT,JUMBO SHOPPER VINTAGE RED PAISLEY,JUMBO STORAGE BAG SUKI,PACK OF 72 RETROSPOT CAKE CASES,PAPER CHAIN KIT 50'S CHRISTMAS
12,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0
14,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
16,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
21,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


## Association Rule Mining with FP Growth

In [24]:
input_assoc_rules = output_df_uk
domain_transaction = Domain([DiscreteVariable.make(name = item,
                                                   values = ['0', '1']) for item in input_assoc_rules.columns])
data_transaction_uk = Orange.data.Table.from_numpy(domain = domain_transaction,
                                                   X = input_assoc_rules.as_matrix(),
                                                   Y = None)
data_transaction_uk_en, mapping = OneHot.encode(data_transaction_uk, 
                                                include_class = False)

In [25]:
min_support = 0.01
print("num of required transactions = ", int(input_assoc_rules.shape[0] * min_support))
num_trans = input_assoc_rules.shape[0] * min_support
itemsets = dict(frequent_itemsets(data_transaction_uk_en, 
                                  min_support = min_support))

num of required transactions =  49


In [26]:
len(itemsets)

645632

In [29]:
# Specify a confidence value and generate rules
confidence = 0.3
rules_df = pd.DataFrame()

if len(itemsets) < 1000000:
    rules = [(P, Q, supp, conf) for P, Q, supp, conf in association_rules(itemsets, confidence) if len(Q) == 1]
    names = {item: '{} = {}'.format(var.name, val) for item, var, val in OneHot.decode(mapping, data_transaction_uk, mapping)}
    
    eligible_ante = [v for k, v in names.items() if v.endswith('1')]
    
    N = input_assoc_rules.shape[0] 
    
    rule_stats = list(rules_stats(rules, itemsets, N))
    
    rule_list_df = []
    for ex_rule_frm_rule_stat in rule_stats:
        ante = ex_rule_frm_rule_stat[0]
        cons = ex_rule_frm_rule_stat[1]
        named_cons = names[next(iter(cons))]
        if named_cons in eligible_ante:
            rule_lhs = [names[i][:-2] for i in ante if names[i] in eligible_ante]
            ante_rule = ', '.join(rule_lhs)
            
            if ante_rule and len(rule_lhs) > 1:
                rule_dict = {'support' : ex_rule_frm_rule_stat[2],
                             'confidence' : ex_rule_frm_rule_stat[3],
                             'converge' : ex_rule_frm_rule_stat[4],
                             'strength': ex_rule_frm_rule_stat[5],
                             'lift': ex_rule_frm_rule_stat[6],
                             'leverage': ex_rule_frm_rule_stat[7],
                             'antecedent': ante_rule,
                             'consequent': named_cons[:-2]}
                rule_list_df.append(rule_dict)
            
    rules_df = pd.DataFrame(rule_list_df)
    print('Raw rules data frame of {} rules generated'. format(rules_df.shape[0]))
    if not rules_df.empty:
        prune_rules_df = rules_df.groupby(['antecedent', 'consequent']).max().reset_index()
    else:
        print('Unable to generate any rule')
    

Raw rules data frame of 117464 rules generated


## Sorting rules in our Grocery Dataset

In [31]:
dw = pd.options.display.max_colwidth
pd.options.display.max_colwidth = 100
prune_rules_df[['antecedent', 
                'consequent', 
                'support', 
                'confidence', 
                'lift']].groupby('consequent') \
                        .max() \
                        .reset_index() \
                        .sort_values(['lift','support','confidence'],
                                     ascending = False).head(5)
                            

,consequent,antecedent,support,confidence,lift
8,PACK OF 72 RETROSPOT CAKE CASES =,"WHITE HANGING HEART T-LIGHT HOLDER =, REGENCY CAKESTAND 3 TIER =, NATURAL SLATE HEART CHALKBOARD =",145,0.971014,5.394404
9,PAPER CHAIN KIT 50'S CHRISTMAS =,"WHITE HANGING HEART T-LIGHT HOLDER =, REGENCY CAKESTAND 3 TIER =, NATURAL SLATE HEART CHALKBOARD =",94,0.597701,4.341428
3,JUMBO SHOPPER VINTAGE RED PAISLEY =,"WHITE HANGING HEART T-LIGHT HOLDER =, PAPER CHAIN KIT 50'S CHRISTMAS =",384,0.879310,4.218819
5,LUNCH BAG BLACK SKULL. =,"WHITE HANGING HEART T-LIGHT HOLDER =, PACK OF 72 RETROSPOT CAKE CASES =, LUNCH BAG RED RETROSPOT =",227,0.852459,4.078157
4,JUMBO STORAGE BAG SUKI =,"WHITE HANGING HEART T-LIGHT HOLDER =, SET OF 3 CAKE TINS PANTRY DESIGN =, JUMBO BAG PINK POLKAD...",405,0.852459,4.016191


In [32]:
pd.options.display.max_colwidth = dw